# 非自洽 HF-GGA 二阶梯度

在这一份笔记中，我们会回顾 HF-GGA 的二阶梯度的推导与实现过程．

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pyscf import scf, gto, lib, grad, hessian, dft
import pyscf.hessian.rks
import pyscf.grad.rks
from functools import partial

import sys
sys.path.append('../../src')
from utilities import val_from_fchk
from hf_helper import HFHelper
from gga_helper import GGAHelper
from numeric_helper import NumericDiff
from ncgga_engine import NCGGAEngine

np.einsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.set_printoptions(5, linewidth=120, suppress=True)

In [ ]:
mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.5  0.0  0.0
H  0.0  0.7  1.5
"""
mol.basis = "6-31G"
mol.verbose = 0
mol.build()
print("---> Hidden Block Info: Object `mol` built")
print("     For original code, refer to `basic_scf.ipynb#PySCF-自洽场计算`")

In [ ]:
nmo = nao = mol.nao
natm = mol.natm
nocc = mol.nelec[0]
nvir = nmo - nocc
so = slice(0, nocc)
sv = slice(nocc, nmo)
sa = slice(0, nmo)
print("---> Hidden Block Info: Molecular related variables built <--")
print("     For original code, refer to `basic_mp2.ipynb#分子相关量定义`")

In [ ]:
grids = dft.gen_grid.Grids(mol)
grids.atom_grid = (99, 590)
grids.becke_scheme = dft.gen_grid.stratmann
grids.build()
print("---> Hidden Block Info: DFT grids built")
print("     For original code, refer to `basic_gga.ipynb#格点定义`")

In [ ]:
def mol_slice(atm_id, mol=mol):
    _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
    return slice(p0, p1)

In [ ]:
hfh = HFHelper(mol)
hfh.get_all()
mol_slice=hfh.mol_slice

In [ ]:
nch = GGAHelper(mol, "b3lypg", grids, init_scf=False)
ncengine = NCGGAEngine(hfh, nch)
ncengine.get_E_1()

这里为了方便，我们把格点信息放在 `grdh` 与 `kerh` 中，并且定义密度变量 `D`．

In [ ]:
grdh = nch.grdh
kerh = nch.kerh
D = hfh.D

## 二阶梯度：数值解

在上一节中，我们已经介绍过 [外部程序](grad_hf_ncgga.ipynb#外部程序) 的计算流程了．这里我们就借用这个程序生成数值二阶梯度．需要注意二阶梯度需要跑一些时间．

In [ ]:
def mol_to_ncgrad(mol):
    hfh = HFHelper(mol)
    nch = GGAHelper(mol, "b3lypg", grids, init_scf=False)
    ncengine = NCGGAEngine(hfh, nch)
    return ncengine.get_E_1()

In [ ]:
%%time
#ncgrad_diff = NumericDiff(mol, mol_to_ncgrad, deriv=2, p5=True, interval=3e-4).get_numdif()

In [ ]:
#ncgrad_diff

## 二阶梯度：外部引擎

在这里，我们先介绍外部引擎做法．一段二阶梯度计算过程已经可以通过简单的代码来实现．注意到我们刚才已经定义过 `ncengine` 变量了：

In [ ]:
%%time
E_2_byengine = ncengine.get_E_2()

In [ ]:
#np.allclose(E_2_byengine, ncgrad_diff, atol=1e-6)

下面我们就来描述这段代码的实际计算过程．

## 二阶梯度：简便解法

我们先简单回顾二阶梯度的计算流程．我们在 [GGA 二阶梯度](grad_gga.ipynb#实现参考：二阶导数性质) 中已经导出下式：

\begin{align}
\frac{\partial^2}{\partial A_t \partial B_s} E_\mathrm{elec}
&= \partial_{A_t} \partial_{B_s} E_\mathrm{elec} + 4 U_{pi}^{A_t} F_{pi}^{B_s, \mathrm{n}} + 4 U_{pi}^{B_s} F_{pi}^{A_t, \mathrm{n}} + 4 U_{pi}^{A_t} U_{qi}^{B_s} F_{pq}^\mathrm{n} + 4 U_{pi}^{A_t B_s} F_{pi}^\mathrm{n} + 4 U_{pi}^{A_t} A_{pi, qj}^\mathrm{n} U_{qj}^{B_s}
\end{align}

我们将上式分为三部分来作程序实现．

`E_SS` $E_\mathrm{elec}^{A_t B_s, \mathrm{SS}} = \partial_{A_t} \partial_{B_s} E_\mathrm{elec}$ 二阶梯度 (Skeleton-Skeleton)

这一段的公式较为复杂，可以参考 [GGA 二阶梯度](grad_gga.ipynb#实现参考：二阶导数性质) 中的说明．事实上，只要将所有涉及泛函格点计算，以及交换积分系数的部分全部替换为非自洽泛函，就可以了．

关于这一段，事实上我们只要稍对 PySCF 的高层函数 `hessian.rks.partial_hess_elec` 作一些修改就可以得到结果；但不能直接利用，因为它还会多计算 $S_{ii}^{A_t B_s} \varepsilon_i$ 的贡献项；这是我们不希望的，因为它应当是包含在 $E_\mathrm{elec}^{A_t B_s, \mathrm{UU}}$ 中的项，且不应用自洽方法的轨道能，而是用非自洽泛函的 Fock 矩阵得到．

In [ ]:
tmp_tensor_1 = (
    + 2 * np.einsum("g, Tgu, gv -> Tuv", kerh.fr, grdh.ao_2T, grdh.ao_0)
    + 4 * np.einsum("g, rg, rTgu, gv -> Tuv", kerh.fg, grdh.rho_1, grdh.ao_3T, grdh.ao_0)
    + 4 * np.einsum("g, rg, Tgu, rgv -> Tuv", kerh.fg, grdh.rho_1, grdh.ao_2T, grdh.ao_1)
)
XX, XY, XZ, YY, YZ, ZZ = range(6)
E_SS_GGA_contrib1 = np.zeros((natm, natm, 3, 3))
for A in range(natm):
    sA = mol_slice(A)
    E_SS_GGA_contrib1[A, A] = np.einsum("Tuv, uv -> T", tmp_tensor_1[:, sA], D[sA])[[XX, XY, XZ, XY, YY, YZ, XZ, YZ, ZZ]].reshape(3, 3)
    
tmp_tensor_2 = 4 * np.einsum("g, rg, trgu, sgv -> tsuv", kerh.fg, grdh.rho_1, grdh.ao_2, grdh.ao_1)
tmp_tensor_2 += tmp_tensor_2.transpose(1, 0, 3, 2)
tmp_tensor_2 += 2 * np.einsum("g, tgu, sgv -> tsuv", kerh.fr, grdh.ao_1, grdh.ao_1)
E_SS_GGA_contrib2 = np.empty((natm, natm, 3, 3))
for A in range(natm):
    sA = mol_slice(A)
    for B in range(A + 1):
        sB = mol_slice(B)
        E_SS_GGA_contrib2[A, B] = np.einsum("tsuv, uv -> ts", tmp_tensor_2[:, :, sA, sB], D[sA, sB])
        if A != B:
            E_SS_GGA_contrib2[B, A] = E_SS_GGA_contrib2[A, B].T
            
E_SS_GGA_contrib3 = (
    + np.einsum("g, Atg, Bsg -> ABts", kerh.frr, grdh.A_rho_1, grdh.A_rho_1)
    + 2 * np.einsum("g, wg, Atwg, Bsg -> ABts", kerh.frg, grdh.rho_1, grdh.A_rho_2, grdh.A_rho_1)
    + 2 * np.einsum("g, Atg, rg, Bsrg -> ABts", kerh.frg, grdh.A_rho_1, grdh.rho_1, grdh.A_rho_2)
    + 4 * np.einsum("g, wg, Atwg, rg, Bsrg -> ABts", kerh.fgg, grdh.rho_1, grdh.A_rho_2, grdh.rho_1, grdh.A_rho_2)
    + 2 * np.einsum("g, Atrg, Bsrg -> ABts", kerh.fg, grdh.A_rho_2, grdh.A_rho_2)
)

E_SS_HF_contrib = (
    + np.einsum("ABtsuv, uv -> ABts", hfh.H_2_ao, D)
    + 0.5 * np.einsum("ABtsuvkl, uv, kl -> ABts", hfh.eri2_ao, D, D)
    - 0.25 * nch.cx * np.einsum("ABtsukvl, uv, kl -> ABts", hfh.eri2_ao, D, D)
)

E_SS = E_SS_GGA_contrib1 + E_SS_GGA_contrib2 + E_SS_GGA_contrib3 + E_SS_HF_contrib

`E_SU` $E_\mathrm{elec}^{A_t B_s, \mathrm{SU}} = \partial_{A_t} \partial_{B_s}^\mathrm{U} E_\mathrm{elec} + \partial_{A_t}^\mathrm{U} \partial_{B_s} E_\mathrm{elec}$ 二阶梯度 (Skeleton-U)

$$
E_\mathrm{elec}^{A_t B_s, \mathrm{SU}} = 4 U_{pi}^{A_t} F_{pi}^{B_s, \mathrm{n}} + 4 U_{pi}^{B_s} F_{pi}^{A_t, \mathrm{n}}
$$

我们只要生成 $F_{\mu \nu}^{A_t, \mathrm{n}} [\rho]$ 即可．这一部分可以参考 [GGA 一阶梯度性质](grad_gga.ipynb#实现参考：一阶导数性质)；事实上，PySCF 的高级函数 `hessian.rks.make_h1` 也能做相同的事．

In [ ]:
nch.F_1_ao = nch.scf_hess.make_h1(hfh.C, hfh.mo_occ)
nch.F_1_mo = hfh.C.T @ nch.F_1_ao @ hfh.C

In [ ]:
E_SU = (
    + 4 * np.einsum("Atpi, Bspi -> ABts", hfh.U_1[:, :, :, so], nch.F_1_mo[:, :, :, so])
    + 4 * np.einsum("Bspi, Atpi -> ABts", hfh.U_1[:, :, :, so], nch.F_1_mo[:, :, :, so])
)

`E_SS` Energy second derivative (U-U)
`E_SS` $E_\mathrm{elec}^{A_t B_s, \mathrm{UU}} = 4 U_{pi}^{A_t} U_{qi}^{B_s} F_{pq}^\mathrm{n} + 4 U_{pi}^{A_t B_s} F_{pi}^\mathrm{n} + 4 U_{pi}^{A_t} A_{pi, qj}^\mathrm{n} U_{qj}^{B_s}$ 二阶梯度 (U-U)

尽管看起来我们已经将上式的所有元素生成了，但注意到 HFHelper 中暂时还没有给出 $U_{ki}^{A_t B_s}$ 的二阶 U 矩阵，因此我们还需要利用 $\xi_{ij}^{A_t B_s} + U_{ij}^{A_t B_s} + U_{ji}^{A_t B_s} = 0$ 的条件：

$$
E_\mathrm{elec}^{A_t B_s, \mathrm{UU}} = 4 U_{pi}^{A_t} U_{qi}^{B_s} F_{pq}^\mathrm{n} + 4 U_{ai}^{A_t B_s} F_{ai}^\mathrm{n} - 2 \xi_{ki}^{A_t B_s} F_{ki}^\mathrm{n} + 4 U_{pi}^{A_t} A_{pi, qj}^\mathrm{n} U_{qj}^{B_s}
$$

In [ ]:
E_UU = (
    + 4 * np.einsum("Atpi, Bsqi, pq -> ABts", hfh.U_1[:, :, :, so], hfh.U_1[:, :, :, so], nch.F_0_mo)
    + 4 * np.einsum("ABtsai, ai -> ABts", hfh.U_2_vo, nch.F_0_mo[sv, so])
    - 2 * np.einsum("ABtski, ki -> ABts", hfh.Xi_2[:, :, :, :, so, so], nch.F_0_mo[so, so])
    + 4 * np.einsum("Atpi, Bspi -> ABts", hfh.U_1[:, :, :, so], nch.Ax0_Core(sa, so, sa, so)(hfh.U_1[:, :, :, so]))
)

我们将几个电子态能量的二阶梯度贡献项相加，并加上原子核的梯度贡献项，就能得到最终的结果．它与数值二阶梯度的误差在 1E-6 Hartree 量级上，应当认为是基本正确的．可能使用其它软件，在不同格点的权重选取 (Becke Scheme) 或者约减 (Prune) 下，误差都会比这个数量级大．这在 [PySCF 一阶、二阶梯度与 Gaussian 梯度对比](grad_gga.ipynb#PySCF-一阶、二阶梯度计算) 中就能看到．

In [ ]:
#np.allclose(
#    E_SS + E_SU + E_UU + nch.scf_hess.hess_nuc(),
#    ncgrad_diff,
#    atol=1e-6
#)

In [ ]:
#abs(E_SS + E_SU + E_UU + nch.scf_hess.hess_nuc() - ncgrad_diff).max()

如果使用 Z-Vector 解法来避免多次二阶 CP-HF 方程，我们可以使用与 [一阶梯度过程](grad_hf_ncgga.ipynb#Z-Vector-方程求解) 完全相同的 Z-Vector $Z_{ai}$，求得 $Z_{ai} B_{ai}^{A_t B_s}$ 替换 $U_{ai}^{A_t B_s} F_{ai}^\mathrm{n}$．这里就不再详述了．

我们在这里不提及普通的 Z-Vector 解法：我理解所谓普通的 Z-Vector 方法是将 $U_{ai}^{A_t B_s} F_{ai}^\mathrm{n}$ 替换为 $Z_{ai} B_{ai}^{A_t B_s}$，从而避免多次二阶 CP-HF 方程的求解．这里使用的 Z-Vector $Z_{ai}$ 实际上与 [一阶梯度过程](grad_hf_ncgga.ipynb#Z-Vector-方程求解) 中使用的 Z-Vector 完全相同，因此借用相同的过程就能得到结果．但是，这并不能解决 $U_{ij}^{A_t}$ 会出现 [数值不稳定](u_rhf.ipynb#实现参考：一阶-U-矩阵) 的情况．因此，我们需要构造另一种 Z-Vector 过程．

## 二阶梯度：安全解法

我们发现上面的解法不能解决 $U_{ij}^{A_t}$ 会出现 [数值不稳定](u_rhf.ipynb#实现参考：一阶-U-矩阵) 的情况．因此，我们需要对上面的公式化简为繁，得到一种不使用 $U_{ij}^{A_t}$ 的解法．当然，我们会允许 $U_{ii}^{A_t} = - \frac{1}{2} S_{ii}^{A_t}$．后文中，我们称 $U_{ij}^{A_t}$ 在 $i \neq j$ 时是不安全的；这对于非占轨道也是相同的．

这里将会出现非常繁琐的公式推导与程序．为了避免数值不稳定的情况，我们指定 `U_1_vo` $U_{ai}^{A_t}$ 与 `U_1_ov` $U_{ia}^{A_t}$．我们最终的程序的目标是将所有出现 `U_1` 的部分全部替换为 `U_1_vo` 或 `U_1_ov`．

为了程序的简便，我们在这里对程序的一些变量作定义：

* `U_1` $U_{pq}^{A_t}$

* `Xi_2` $\xi_{pq}^{A_t B_s}$

* `Ax0_Core` $A_{pq, rs}$；注意我们还有 `nch.Ax0_Core` 指代 $A_{pq, rs}^\mathrm{n}$

* `Ax1_Core` $A_{pq, rs}^{A_t}$；我们不需使用 `nch.Ax1_Core`

* `F_1_mo` $F_{pq}^{A_t}$；注意我们还有 `nch.F_1_mo` 指代 $F_{pq}^{A_t, \mathrm{n}}$

* `S_1_mo` $S_{pq}^{A_t}$

* `F_2_ao` $F_{\mu \nu}^{A_t B_s}$；我们不需使用 `nch.F_2_ao`

* `S_2_mo` $S_{pq}^{A_t B_s}$

* `e, eo, ev` $\varepsilon_p$ 轨道能；对于非自洽泛函不存在该概念

In [ ]:
U_1 = hfh.U_1
Xi_2 = hfh.Xi_2
Ax0_Core = hfh.Ax0_Core
Ax1_Core = hfh.Ax1_Core
F_1_mo = hfh.F_1_mo
S_1_mo = hfh.S_1_mo
F_2_ao = hfh.F_2_ao
S_2_mo = hfh.S_2_mo
e, eo, ev = hfh.e, hfh.eo, hfh.ev

In [ ]:
U_1_vo = U_1[:, :, sv, so]
U_1_ov = U_1[:, :, so, sv]

### Skeleton-U 部分

不安全的 Skeleton-U 部分为

$$
E_\mathrm{elec}^{A_t B_s, \mathrm{SU}} = 4 U_{pi}^{A_t} F_{pi}^{B_s, \mathrm{n}} + 4 U_{pi}^{B_s} F_{pi}^{A_t, \mathrm{n}}
$$

安全的 Skeleton-U 非常容易获得．我们注意到 $F_{ki}^{A_t, \mathrm{n}}$ 关于角标 $k, i$ 对称，因此我们能利用 $S_{ki}^{A_t} + U_{ki}^{A_t} + U_{ik}^{A_t} = 0$．这类技巧我们以后就不加说明了．

$$
E_\mathrm{elec}^{A_t B_s, \mathrm{SU}} = 4 U_{ai}^{A_t} F_{ai}^{B_s, \mathrm{n}} + 4 U_{ai}^{B_s} F_{ai}^{A_t, \mathrm{n}} - 2 U_{ki}^{A_t} F_{ki}^{B_s, \mathrm{n}} - 2 U_{ki}^{B_s} F_{ki}^{A_t, \mathrm{n}}
$$

In [ ]:
E_SU_safe = (
    + 4 * np.einsum("Atpi, Bspi -> ABts", U_1_vo, nch.F_1_mo[:, :, sv, so])
    + 4 * np.einsum("Bspi, Atpi -> ABts", U_1_vo, nch.F_1_mo[:, :, sv, so])
    - 2 * np.einsum("Atpi, Bspi -> ABts", S_1_mo[:, :, so, so], nch.F_1_mo[:, :, so, so])
    - 2 * np.einsum("Bspi, Atpi -> ABts", S_1_mo[:, :, so, so], nch.F_1_mo[:, :, so, so])
)
np.allclose(E_SU_safe, E_SU)

当然，我们还可以稍稍利用 $A_t, B_s$ 的对称性；不过这里不是计算量的大头，因此不算太重要，只是代码看起来更干练一些．

In [ ]:
E_SU_safe = (
    + 4 * np.einsum("Atpi, Bspi -> ABts", U_1_vo, nch.F_1_mo[:, :, sv, so])
    - 2 * np.einsum("Atpi, Bspi -> ABts", S_1_mo[:, :, so, so], nch.F_1_mo[:, :, so, so])
)
E_SU_safe += E_SU_safe.transpose(1, 0, 3, 2)
np.allclose(E_SU_safe, E_SU)

### U-U 部分：四脚标项

不安全的 U-U 贡献是

$$
E_\mathrm{elec}^{A_t B_s, \mathrm{UU}} = 4 U_{pi}^{A_t} U_{qi}^{B_s} F_{pq}^\mathrm{n} + 4 U_{ai}^{A_t B_s} F_{ai}^\mathrm{n} - 2 \xi_{ij}^{A_t B_s} F_{ij}^\mathrm{n} + 4 U_{pi}^{A_t} A_{pi, qj}^\mathrm{n} U_{qj}^{B_s}
$$

U-U 的贡献相对麻烦，我们先从简单的入手．这一部分的目标是考察

$$
4 U_{pi}^{A_t} A_{pi, qj}^\mathrm{n} U_{qj}^{B_s}
$$

这一项是完全安全的：

\begin{align}
4 U_{pi}^{A_t} A_{pi, qj}^\mathrm{n} U_{qj}^{B_s} &=
4 U_{ai}^{A_t} A_{ai, bj}^\mathrm{n} U_{bj}^{B_s} +
4 U_{ki}^{A_t} A_{ki, bj}^\mathrm{n} U_{bj}^{B_s} +
4 U_{ai}^{A_t} A_{ai, lj}^\mathrm{n} U_{lj}^{B_s} +
4 U_{ki}^{A_t} A_{ki, lj}^\mathrm{n} U_{lj}^{B_s} \\ &=
4 U_{ai}^{A_t} A_{ai, bj}^\mathrm{n} U_{bj}^{B_s} -
2 S_{ki}^{A_t} A_{ki, bj}^\mathrm{n} U_{bj}^{B_s} -
2 U_{ai}^{A_t} A_{ai, lj}^\mathrm{n} S_{lj}^{B_s} +
S_{ki}^{A_t} A_{ki, lj}^\mathrm{n} S_{lj}^{B_s}
\end{align}

In [ ]:
E_UU_safe_1 = (
    + 4 * np.einsum("Atai, Bsai -> ABts", U_1_vo, nch.Ax0_Core(sv, so, sv, so)(U_1_vo))
    - 2 * np.einsum("Atki, Bski -> ABts", S_1_mo[:, :, so, so], nch.Ax0_Core(so, so, sv, so)(U_1_vo))
    - 2 * np.einsum("Atai, Bsai -> ABts", U_1_vo, nch.Ax0_Core(sv, so, so, so)(S_1_mo[:, :, so, so]))
    + 1 * np.einsum("Atki, Bski -> ABts", S_1_mo[:, :, so, so], nch.Ax0_Core(so, so, so, so)(S_1_mo[:, :, so, so]))
)
np.allclose(
    E_UU_safe_1,
    4 * np.einsum("Atpi, Bspi -> ABts", U_1[:, :, :, so], nch.Ax0_Core(sa, so, sa, so)(U_1[:, :, :, so]))
)

不过，我们指出，由于 `A_{pi, qj}^\mathrm{n}` 是 GGA 的张量缩并，其计算耗时还是比较大的，因此我们这里希望能避免这类计算．我们可以对时间作一个估计：使用 `A_{pi, qj}^\mathrm{n}` 的函数 `nch.Ax0_Core`，服务器主节点在 $X_{pi}$ 上的计算耗时是 60 ms 左右；而我们现在的体系大小是 4 个原子，因此需要 4 * 3 * 60 ms = 720 ms 的接近 1 s 的时间．对这部分的计算量我们能省则省．

一类通常的做法是定义一个新的矩阵 `U_pi_fake` $\mathscr{U}_{pi}^{A_t}$：

\begin{align}
\mathscr{U}_{ai}^{A_t} &= U_{ai}^{A_t} \\
\mathscr{U}_{ki}^{A_t} &= - \frac{1}{2} S_{ki}^{A_t}
\end{align}

因此，

$$
4 U_{pi}^{A_t} A_{pi, qj}^\mathrm{n} U_{qj}^{B_s} = 4 \mathscr{U}_{pi}^{A_t} A_{pi, qj}^\mathrm{n} \mathscr{U}_{qj}^{B_s}
$$

但需要强调，这只是为了计算效率而定义的新的变量；它不能代替真正的 $U_{pi}^{A_t}$．

In [ ]:
U_pi_fake = np.empty((natm, 3, nmo, nocc))
U_pi_fake[:, :, so, so] = - 0.5 * S_1_mo[:, :, so, so]
U_pi_fake[:, :, sv, so] = U_1_vo

In [ ]:
E_UU_safe_1 = 4 * np.einsum("Atpi, Bspi -> ABts", U_pi_fake, nch.Ax0_Core(sa, so, sa, so)(U_pi_fake))
np.allclose(
    E_UU_safe_1,
    4 * np.einsum("Atpi, Bspi -> ABts", U_1[:, :, :, so], nch.Ax0_Core(sa, so, sa, so)(U_1[:, :, :, so]))
)

### U-U 部分：$4 U_{pi}^{A_t} U_{qi}^{B_s} F_{pq}^\mathrm{n}$ 安全项

接下来的工作就是非常枯燥的工作了．我们将会对剩余的三项作安全部分与不安全部分的分离；最终将不安全的部分相加，得到安全的结果．首先我们处理 $4 U_{pi}^{A_t} U_{qi}^{B_s} F_{pq}^\mathrm{n}$．

\begin{align*}
  4 U_{pi}^{A_t} U_{qi}^{B_s} F_{pq}^\mathrm{n}
  &=
  4 U_{ai}^{A_t} U_{qi}^{B_s} F_{aq}^\mathrm{n} + 4 U_{ki}^{A_t} U_{qi}^{B_s} F_{kq}^\mathrm{n} \\
  &=
  4 U_{ai}^{A_t} U_{bi}^{B_s} F_{ab}^\mathrm{n} + 4 U_{ki}^{A_t} U_{bi}^{B_s} F_{kb}^\mathrm{n} +
  4 U_{ai}^{A_t} U_{li}^{B_s} F_{al}^\mathrm{n} + 4 U_{ki}^{A_t} U_{li}^{B_s} F_{kl}^\mathrm{n} \\
\end{align*}

我们会发现，上式中真正安全的只有 $4 U_{ai}^{A_t} U_{bi}^{B_s} F_{ab}^\mathrm{n}$．剩余的项都是不安全的．

<div class="alert alert-info">

**任务**

1. 验证对 $k, b, i$ 求和的 $4 U_{ki}^{A_t} U_{bi}^{B_s} F_{kb}^\mathrm{n} \neq - 2 S_{ki}^{A_t} U_{bi}^{B_s} F_{kb}^\mathrm{n}$．请从对称性的角度说明为何不等号通常成立．请验证剩余两项也有不等号成立．

   我们必须在这个问题上有所警惕，因为以后需要经常判断一些表达式是否可以从不安全化为安全．在这个问题上，动手写代码来验证通常比动脑推导更不容易出错．

</div>

因此，我们将这部分的安全项提取出来：

In [ ]:
E_UU_safe_2 = 4 * np.einsum("Atai, Bsbi, ab -> ABts", U_1_vo, U_1_vo, nch.F_0_mo[sv, sv])

剩余的不安全项是

* $4 U_{ki}^{A_t} U_{bi}^{B_s} F_{kb}^\mathrm{n}$

* $4 U_{ai}^{A_t} U_{li}^{B_s} F_{al}^\mathrm{n}$

* $4 U_{ki}^{A_t} U_{li}^{B_s} F_{kl}^\mathrm{n}$

### U-U 部分：$- 2 \xi_{ij}^{A_t B_s} F_{ij}^\mathrm{n}$ 安全项

首先我们回顾 $\xi_{ij}^{A_t B_s}$ 的表达式：

Yamaguchi (p405, L.4)

\begin{align*}
  \xi_{ij}^{A_t B_s}
  &= \frac{1}{2} S_{ij}^{A_t B_s} + U_{im}^{A_t} U_{jm}^{B_s} - S_{im}^{A_t} S_{jm}^{B_s} + \mathrm{interchange} (A_t, B_s) \\
  &= \frac{1}{2} S_{ij}^{A_t B_s} + U_{ik}^{A_t} U_{jk}^{B_s} + U_{ia}^{A_t} U_{ja}^{B_s} - S_{im}^{A_t} S_{jm}^{B_s}
  + \mathrm{interchange} (A_t, B_s)
\end{align*}

这里我们用对称性简化了表达．因此，我们有安全项

$$
(- S_{ij}^{A_t B_s} - 2 U_{ia}^{A_t} U_{ja}^{B_s} + 2 S_{im}^{A_t} S_{jm}^{B_s}) F_{ij}^\mathrm{n} + \mathrm{interchange} (A_t, B_s)
$$

In [ ]:
E_UU_safe_3 = (
    - 1 * np.einsum("ABtsij, ij -> ABts", S_2_mo[:, :, :, :, so, so], nch.F_0_mo[so, so])
    - 2 * np.einsum("Atia, Bsja, ij -> ABts", U_1_ov, U_1_ov, nch.F_0_mo[so, so])
    + 2 * np.einsum("Atim, Bsjm, ij -> ABts", S_1_mo[:, :, so, :], S_1_mo[:, :, so, :], nch.F_0_mo[so, so])
)
E_UU_safe_3 += E_UU_safe_3.transpose(1, 0, 3, 2)

而不安全的项则是 (注意到下式利用了 $i, j$ 求和情况下的对称性)

* $-4 U_{ik}^{A_t} U_{jk}^{B_s} F_{ij}^\mathrm{n}$

我们会发现不安全项 $-4 U_{ik}^{A_t} U_{jk}^{B_s} F_{ij}^\mathrm{n}$ 与前面的 $4 U_{ki}^{A_t} U_{li}^{B_s} F_{kl}^\mathrm{n}$ 是可以抵消的．

我们重新写一下现在剩余的不安全项：

* $4 U_{ak}^{B_s} U_{ik}^{A_t} F_{ai}^\mathrm{n}$

* $4 U_{ak}^{A_t} U_{ik}^{B_s} F_{ai}^\mathrm{n}$

我们会发现，这些项与最后的 U-U 部分 $4 U_{ai}^{A_t B_s} F_{ai}^\mathrm{n}$ 共享 $F_{ai}^\mathrm{n}$．

### U-U 部分：剩余项的 Z-Vector 方程

我们现在构造一种 Z-Vector 解法．我们现在剩下的项是

$$
4 (U_{ai}^{A_t B_s} + U_{ak}^{A_t} U_{ik}^{B_s} + U_{ak}^{B_s} U_{ik}^{A_t}) F_{ai}^\mathrm{n}
$$

像往常一样，我们定义 $A_{ai, bj}' = - A_{ai, bj} - \delta_{ab} \delta_{ij} (\varepsilon_b - \varepsilon_j))$．如果我们再定义

$$
\mathscr{B}_{ai}^{A_t B_s} = A_{ai, bj}' (U_{bj}^{A_t B_s} + U_{bk}^{A_t} U_{jk}^{B_s} + U_{bk}^{B_s} U_{jk}^{A_t})
$$

以及 Z-Vector

$$
A'_{ai, bj} Z_{bj} = F_{ai}^\mathrm{n}
$$

那么

$$
4 (U_{ai}^{A_t B_s} + U_{ak}^{A_t} U_{ik}^{B_s} + U_{ak}^{B_s} U_{ik}^{A_t}) F_{ai}^\mathrm{n} = Z_{bj} \mathscr{B}_{ai}^{A_t B_s}
$$

那么现在的最重要任务是构建 $\mathscr{B}_{ai}^{A_t B_s}$，因为 Z-Vector 已经是非常容易得到的了：

In [ ]:
Z = scf.cphf.solve(Ax0_Core(sv, so, sv, so), hfh.e, hfh.mo_occ, nch.F_0_mo[sv, so])[0]

构建 $\mathscr{B}_{ai}^{A_t B_s}$ 是一个非常痛苦的过程．我们需要一步一步来．

#### $B_{ai}^{A_t B_s}$ 安全项分析

$\mathscr{B}_{ai}^{A_t B_s}$ 的一个重要贡献项显然是 $A_{ai, bj}' U_{bj}^{A_t B_s} = B_{ai}^{A_t B_s}$．这里我们首先要将 $B_{ai}^{A_t B_s}$ 的所有项拆开分析．

Yamaguchi (p437-438, X.5)

\begin{align}
B_{ai}^{A_t B_s} &= F_{ai}^{A_t B_s} - \xi_{ai}^{A_t B_s} \varepsilon_i - \frac{1}{2} A_{ai, kl} \xi_{kl}^{A_t B_s} \\
&\quad\mathrel+ U_{pa}^{A_t} F_{pi}^{B_s} + U_{pa}^{B_s} F_{pi}^{A_t} + U_{pi}^{A_t} F_{pa}^{B_s} + U_{pi}^{B_s} F_{pa}^{A_t} \\
&\quad\mathrel+ U_{pa}^{A_t} U_{pi}^{B_s} \varepsilon_p + U_{pa}^{B_s} U_{pi}^{A_t} \varepsilon_p \\
&\quad\mathrel+ A_{ai, pq} U_{pk}^{A_t} U_{qk}^{B_s} \\
&\quad\mathrel+ U_{pa}^{A_t} A_{pi, qk} U_{qk}^{B_s} + U_{pa}^{B_s} A_{pi, qk} U_{qk}^{A_t} \\
&\quad\mathrel+ U_{pi}^{A_t} A_{pa, qk} U_{qk}^{B_s} + U_{pi}^{B_s} A_{pa, qk} U_{qk}^{A_t} \\
&\quad\mathrel+ A_{ai, pk}^{A_t} U_{pk}^{B_s} + A_{ai, pk}^{B_s} U_{pk}^{A_t}
\end{align}

In [ ]:
B_2_vo_ = (
    # line 1
    np.einsum("ABtsuv, ua, vi -> ABtsai", hfh.F_2_ao, hfh.Cv, hfh.Co)
    - np.einsum("ABtsai, i -> ABtsai", Xi_2[:, :, :, :, sv, so], eo)
    - 0.5 * Ax0_Core(sv, so, so, so)(Xi_2[:, :, :, :, so, so])
    # line 2
    + np.einsum("Atpa, Bspi -> ABtsai", U_1[:, :, :, sv], F_1_mo[:, :, :, so])
    + np.einsum("Bspa, Atpi -> ABtsai", U_1[:, :, :, sv], F_1_mo[:, :, :, so])
    + np.einsum("Atpi, Bspa -> ABtsai", U_1[:, :, :, so], F_1_mo[:, :, :, sv])
    + np.einsum("Bspi, Atpa -> ABtsai", U_1[:, :, :, so], F_1_mo[:, :, :, sv])
    # line 3
    + np.einsum("Atpa, Bspi, p -> ABtsai", U_1[:, :, :, sv], U_1[:, :, :, so], e)
    + np.einsum("Bspa, Atpi, p -> ABtsai", U_1[:, :, :, sv], U_1[:, :, :, so], e)
    # line 4
    + Ax0_Core(sv, so, sa, sa)(np.einsum("Atkm, Bslm -> ABtskl", U_1[:, :, :, so], U_1[:, :, :, so]))
    # line 5
    + np.einsum("Atpa, Bspi -> ABtsai", U_1[:, :, :, sv], Ax0_Core(sa, so, sa, so)(U_1[:, :, :, so]))
    + np.einsum("Bspa, Atpi -> ABtsai", U_1[:, :, :, sv], Ax0_Core(sa, so, sa, so)(U_1[:, :, :, so]))
    # line 6
    + np.einsum("Atpi, Bspa -> ABtsai", U_1[:, :, :, so], Ax0_Core(sa, sv, sa, so)(U_1[:, :, :, so]))
    + np.einsum("Bspi, Atpa -> ABtsai", U_1[:, :, :, so], Ax0_Core(sa, sv, sa, so)(U_1[:, :, :, so]))
    # line 7
    + Ax1_Core(sv, so, sa, so)(U_1[:, :, :, so])
    + Ax1_Core(sv, so, sa, so)(U_1[:, :, :, so]).transpose((1, 0, 3, 2, 4, 5))
)
np.allclose(B_2_vo_, hfh.B_2_vo)

我们一行一行来．

**第一行**

\begin{align*}
  &\quad F_{ai}^{A_t B_s} - \xi_{ai}^{A_t B_s} \varepsilon_i - \frac{1}{2} A_{ai, kl} \xi_{kl}^{A_t B_s} \\
  &= \mathrm{(Safe)} \\
  % term 1
  &\quad \mathrel+
  F_{ai}^{A_t B_s}
  % term 2
  - S_{ai}^{A_t B_s} \varepsilon_i
  + S_{im}^{A_t} S_{am}^{B_s} \varepsilon_i
  + S_{im}^{B_s} S_{am}^{A_t} \varepsilon_i
  % term 3
  - \frac{1}{2} A_{ai, kl} S_{kl}^{A_t B_s}
  - A_{ai, kl} S_{km}^{A_t} S_{lm}^{B_s}
  - A_{ai, kl} U_{kb}^{A_t} U_{lb}^{B_s} \\
  &\quad \mathrm{(Unsafe)} \\
  % term 2
  &\quad \mathrel-
    U_{ik}^{A_t} U_{ak}^{B_s} \varepsilon_i
  - U_{ib}^{A_t} U_{ab}^{B_s} \varepsilon_i
  - U_{ik}^{B_s} U_{ak}^{A_t} \varepsilon_i
  - U_{ib}^{B_s} U_{ab}^{A_t} \varepsilon_i
  % term 3
  - A_{ai, kl} U_{kj}^{A_t} U_{lj}^{B_s}
\end{align*}

In [ ]:
np.allclose(
# -- Safe
    # term 1
    np.einsum("ABtsuv, ua, vi -> ABtsai", hfh.F_2_ao, hfh.Cv, hfh.Co)
    # term 2
    - np.einsum("ABtsai, i -> ABtsai", S_2_mo[:, :, :, :, sv, so], eo)
    + np.einsum("Atim, Bsam, i -> ABtsai", S_1_mo[:, :, so, :], S_1_mo[:, :, sv, :], eo)
    + np.einsum("Bsim, Atam, i -> ABtsai", S_1_mo[:, :, so, :], S_1_mo[:, :, sv, :], eo)
    # term 3
    - 0.5 * Ax0_Core(sv, so, so, so)(S_2_mo[:, :, :, :, so, so])
    + Ax0_Core(sv, so, so, so)(np.einsum("Atkm, Bslm -> ABtskl", S_1_mo[:, :, so, :], S_1_mo[:, :, so, :]))
    - Ax0_Core(sv, so, so, so)(np.einsum("Atkb, Bslb -> ABtskl", U_1_ov, U_1_ov))
    # term 3
# -- Unsafe
    # term 2
    - np.einsum("Atik, Bsak, i -> ABtsai", U_1[:, :, so, so], U_1[:, :, sv, so], eo)
    - np.einsum("Atib, Bsab, i -> ABtsai", U_1[:, :, so, sv], U_1[:, :, sv, sv], eo)
    - np.einsum("Bsik, Atak, i -> ABtsai", U_1[:, :, so, so], U_1[:, :, sv, so], eo)
    - np.einsum("Bsib, Atab, i -> ABtsai", U_1[:, :, so, sv], U_1[:, :, sv, sv], eo)
    # term 3
    - Ax0_Core(sv, so, so, so)(np.einsum("Atkj, Bslj -> ABtskl", U_1[:, :, so, so], U_1[:, :, so, so]))
    ,
# -- line 1 origin
    np.einsum("ABtsuv, ua, vi -> ABtsai", hfh.F_2_ao, hfh.Cv, hfh.Co)
    - np.einsum("ABtsai, i -> ABtsai", Xi_2[:, :, :, :, sv, so], eo)
    - 0.5 * Ax0_Core(sv, so, so, so)(Xi_2[:, :, :, :, so, so])
)

我们对不安全项进行标号：

* `01` $- U_{ik}^{A_t} U_{ak}^{B_s} \varepsilon_i$

* `02` $- U_{ib}^{A_t} U_{ab}^{B_s} \varepsilon_i$

* `03` $- U_{ik}^{B_s} U_{ak}^{A_t} \varepsilon_i$

* `04` $- U_{ib}^{B_s} U_{ab}^{A_t} \varepsilon_i$

* `05` $- A_{ai, kl} U_{kj}^{A_t} U_{lj}^{B_s}$

**第二行**

\begin{align*}
  &\quad U_{pa}^{A_t} F_{pi}^{B_s} + U_{pa}^{B_s} F_{pi}^{A_t} + U_{pi}^{A_t} F_{pa}^{B_s} + U_{pi}^{B_s} F_{pa}^{A_t} \\
  &= \mathrm{(Safe)} \\
  &\quad \mathrel+
    U_{ka}^{A_t} F_{ki}^{B_s}
  + U_{bi}^{A_t} F_{ba}^{B_s}
  + U_{ka}^{B_s} F_{ki}^{A_t}
  + U_{bi}^{B_s} F_{ba}^{A_t}
  \\
  &\quad \mathrm{(Unsafe)} \\
  &\quad \mathrel+
    U_{ba}^{A_t} F_{bi}^{B_s}
  + U_{ki}^{A_t} F_{ka}^{B_s}
  + U_{ba}^{B_s} F_{bi}^{A_t}
  + U_{ki}^{B_s} F_{ka}^{A_t}
\end{align*}

In [ ]:
np.allclose(
# Safe
    + np.einsum("Atka, Bski -> ABtsai", U_1_ov, F_1_mo[:, :, so, so])
    + np.einsum("Atbi, Bsba -> ABtsai", U_1_vo, F_1_mo[:, :, sv, sv])
    + np.einsum("Bska, Atki -> ABtsai", U_1_ov, F_1_mo[:, :, so, so])
    + np.einsum("Bsbi, Atba -> ABtsai", U_1_vo, F_1_mo[:, :, sv, sv])
# Unsafe
    + np.einsum("Atba, Bsbi -> ABtsai", U_1[:, :, sv, sv], F_1_mo[:, :, sv, so])
    + np.einsum("Atki, Bska -> ABtsai", U_1[:, :, so, so], F_1_mo[:, :, so, sv])
    + np.einsum("Bsba, Atbi -> ABtsai", U_1[:, :, sv, sv], F_1_mo[:, :, sv, so])
    + np.einsum("Bski, Atka -> ABtsai", U_1[:, :, so, so], F_1_mo[:, :, so, sv])
    ,
# -- line 2 origin
    + np.einsum("Atpa, Bspi -> ABtsai", U_1[:, :, :, sv], F_1_mo[:, :, :, so])
    + np.einsum("Bspa, Atpi -> ABtsai", U_1[:, :, :, sv], F_1_mo[:, :, :, so])
    + np.einsum("Atpi, Bspa -> ABtsai", U_1[:, :, :, so], F_1_mo[:, :, :, sv])
    + np.einsum("Bspi, Atpa -> ABtsai", U_1[:, :, :, so], F_1_mo[:, :, :, sv])
)

对不安全项进行标号：

* `06` $U_{ba}^{A_t} F_{bi}^{B_s}$

* `07` $U_{ki}^{A_t} F_{ka}^{B_s}$

* `08` $U_{ba}^{B_s} F_{bi}^{A_t}$

* `09` $U_{ki}^{B_s} F_{ka}^{A_t}$

**第三行**

\begin{align*}
  &\quad U_{pa}^{A_t} U_{pi}^{B_s} \varepsilon_p + U_{pa}^{B_s} U_{pi}^{A_t} \varepsilon_p \\
  &= \mathrm{(Unsafe)} \\
  &\quad \mathrel+
    U_{ba}^{A_t} U_{bi}^{B_s} \varepsilon_b
  + U_{ka}^{A_t} U_{ki}^{B_s} \varepsilon_k
  + U_{ba}^{B_s} U_{bi}^{A_t} \varepsilon_b
  + U_{ka}^{B_s} U_{ki}^{A_t} \varepsilon_k
\end{align*}

第三行没有安全项．

In [ ]:
np.allclose(
# Safe: None
# Unsafe
    + np.einsum("Atba, Bsbi, b -> ABtsai", U_1[:, :, sv, sv], U_1[:, :, sv, so], ev)
    + np.einsum("Atka, Bski, k -> ABtsai", U_1[:, :, so, sv], U_1[:, :, so, so], eo)
    + np.einsum("Bsba, Atbi, b -> ABtsai", U_1[:, :, sv, sv], U_1[:, :, sv, so], ev)
    + np.einsum("Bska, Atki, k -> ABtsai", U_1[:, :, so, sv], U_1[:, :, so, so], eo)
    ,
# -- line 3 origin
    + np.einsum("Atpa, Bspi, p -> ABtsai", U_1[:, :, :, sv], U_1[:, :, :, so], e)
    + np.einsum("Bspa, Atpi, p -> ABtsai", U_1[:, :, :, sv], U_1[:, :, :, so], e)
)

对不安全项进行标号：

* `10` $U_{ba}^{A_t} U_{bi}^{B_s} \varepsilon_b$

* `11` $U_{ka}^{A_t} U_{ki}^{B_s} \varepsilon_k$

* `12` $U_{ba}^{B_s} U_{bi}^{A_t} \varepsilon_b$

* `13` $U_{ka}^{B_s} U_{ki}^{A_t} \varepsilon_k$

**第四行**

\begin{align*}
  &\quad A_{ai, pq} U_{pk}^{A_t} U_{qk}^{B_s} \\
  &= \mathrm{(Safe)} \\
  &\quad \mathrel+
    A_{ai, bc} U_{bk}^{A_t} U_{ck}^{B_s}
  \\
  &\quad \mathrm{(Unsafe)} \\
  &\quad \mathrel+
    A_{ai, jb} U_{jk}^{A_t} U_{bk}^{B_s}
  + A_{ai, bj} U_{bk}^{A_t} U_{jk}^{B_s}
  + A_{ai, kl} U_{kj}^{A_t} U_{lj}^{B_s}
\end{align*}

In [ ]:
np.allclose(
# Safe
    + Ax0_Core(sv, so, sv, sv)(np.einsum("Atbk, Bsck -> ABtsbc", U_1_vo, U_1_vo))
# Unsafe
    + Ax0_Core(sv, so, so, sv)(np.einsum("Atjk, Bsbk -> ABtsjb", U_1[:, :, so, so], U_1[:, :, sv, so]))
    + Ax0_Core(sv, so, sv, so)(np.einsum("Atbk, Bsjk -> ABtsbj", U_1[:, :, sv, so], U_1[:, :, so, so]))
    + Ax0_Core(sv, so, so, so)(np.einsum("Atkj, Bslj -> ABtskl", U_1[:, :, so, so], U_1[:, :, so, so]))
    ,
# -- line 4 origin
    + Ax0_Core(sv, so, sa, sa)(np.einsum("Atkm, Bslm -> ABtskl", U_1[:, :, :, so], U_1[:, :, :, so]))
)

对不安全项进行标号：

* `14` $A_{ai, jb} U_{jk}^{A_t} U_{bk}^{B_s}$

* `15` $A_{ai, bj} U_{bk}^{A_t} U_{jk}^{B_s}$

* `16` $A_{ai, kl} U_{kj}^{A_t} U_{lj}^{B_s}$

**第五行**

\begin{align*}
  &\quad U_{pa}^{A_t} A_{pi, qk} U_{qk}^{B_s} + U_{pa}^{B_s} A_{pi, qk} U_{qk}^{A_t} \\
  &= \mathrm{(Safe)} \\
  &\quad \mathrel+
    U_{ka}^{A_t} A_{ki, qk} \mathscr{U}_{qk}^{B_s}
  + U_{ka}^{B_s} A_{ki, qk} \mathscr{U}_{qk}^{A_t}
  \\
  &\quad \mathrm{(Unsafe)} \\
  &\quad \mathrel+
    U_{ba}^{A_t} A_{bi, qk} U_{qk}^{B_s}
  + U_{ba}^{B_s} A_{bi, qk} U_{qk}^{A_t}
\end{align*}

In [ ]:
np.allclose(
# Safe
    + np.einsum("Atka, Bski -> ABtsai", U_1_ov, Ax0_Core(so, so, sa, so)(U_pi_fake))
    + np.einsum("Bska, Atki -> ABtsai", U_1_ov, Ax0_Core(so, so, sa, so)(U_pi_fake))
# Unsafe
    + np.einsum("Atba, Bsbi -> ABtsai", U_1[:, :, sv, sv], Ax0_Core(sv, so, sa, so)(U_1[:, :, :, so]))
    + np.einsum("Bsba, Atbi -> ABtsai", U_1[:, :, sv, sv], Ax0_Core(sv, so, sa, so)(U_1[:, :, :, so]))
    ,
# -- line 5 origin
    + np.einsum("Atpa, Bspi -> ABtsai", U_1[:, :, :, sv], Ax0_Core(sa, so, sa, so)(U_1[:, :, :, so]))
    + np.einsum("Bspa, Atpi -> ABtsai", U_1[:, :, :, sv], Ax0_Core(sa, so, sa, so)(U_1[:, :, :, so]))
)

对不安全项进行编号：

* `17` $U_{ba}^{A_t} A_{bi, qk} U_{qk}^{B_s}$

* `18` $U_{ba}^{B_s} A_{bi, qk} U_{qk}^{A_t}$

**第六行**

\begin{align*}
  &\quad U_{pi}^{A_t} A_{pa, qk} U_{qk}^{B_s} + U_{pi}^{B_s} A_{pa, qk} U_{qk}^{A_t} \\
  &= \mathrm{(Safe)} \\
  &\quad \mathrel+
    U_{bi}^{A_t} A_{ba, qk} \mathscr{U}_{qk}^{B_s}
  + U_{bi}^{B_s} A_{ba, qk} \mathscr{U}_{qk}^{A_t}
  \\
  &\quad \mathrm{(Unsafe)} \\
  &\quad \mathrel+
    U_{ki}^{A_t} A_{ka, qk} U_{qk}^{B_s}
  + U_{ki}^{B_s} A_{ka, qk} U_{qk}^{A_t}
\end{align*}

In [ ]:
np.allclose(
# Safe
    + np.einsum("Atbi, Bsba -> ABtsai", U_1_vo, Ax0_Core(sv, sv, sa, so)(U_pi_fake))
    + np.einsum("Bsbi, Atba -> ABtsai", U_1_vo, Ax0_Core(sv, sv, sa, so)(U_pi_fake))
# Unsafe
    + np.einsum("Atki, Bska -> ABtsai", U_1[:, :, so, so], Ax0_Core(so, sv, sa, so)(U_1[:, :, :, so]))
    + np.einsum("Bski, Atka -> ABtsai", U_1[:, :, so, so], Ax0_Core(so, sv, sa, so)(U_1[:, :, :, so]))
    ,
# -- line 6 origin
    + np.einsum("Atpi, Bspa -> ABtsai", U_1[:, :, :, so], Ax0_Core(sa, sv, sa, so)(U_1[:, :, :, so]))
    + np.einsum("Bspi, Atpa -> ABtsai", U_1[:, :, :, so], Ax0_Core(sa, sv, sa, so)(U_1[:, :, :, so]))
)

对不安全项进行编号：

* `19` $U_{ki}^{A_t} A_{ka, qk} U_{qk}^{B_s}$

* `20` $U_{ki}^{B_s} A_{ka, qk} U_{qk}^{A_t}$

**第七行**

\begin{align*}
  &\quad A_{ai, pk}^{A_t} U_{pk}^{B_s} + A_{ai, pk}^{B_s} U_{pk}^{A_t} \\
  &= \mathrm{(Safe)} \\
  &\quad \mathrel+
    A_{ai, pk}^{A_t} \mathscr{U}_{pk}^{B_s} + A_{ai, pk}^{B_s} \mathscr{U}_{pk}^{A_t}
\end{align*}

In [ ]:
np.allclose(
# Safe
    + Ax1_Core(sv, so, sa, so)(U_pi_fake)
    + Ax1_Core(sv, so, sa, so)(U_pi_fake).transpose((1, 0, 3, 2, 4, 5))
    ,
# -- line 7 origin
    + Ax1_Core(sv, so, sa, so)(U_1[:, :, :, so])
    + Ax1_Core(sv, so, sa, so)(U_1[:, :, :, so]).transpose((1, 0, 3, 2, 4, 5))
)

**初步总结**

我们现在用程序初步总结一下刚才分离出来的安全与非安全项：

In [ ]:
np.allclose(
# -- Safe
    # line 1
    np.einsum("ABtsuv, ua, vi -> ABtsai", hfh.F_2_ao, hfh.Cv, hfh.Co)
    - np.einsum("ABtsai, i -> ABtsai", S_2_mo[:, :, :, :, sv, so], eo)
    + np.einsum("Atim, Bsam, i -> ABtsai", S_1_mo[:, :, so, :], S_1_mo[:, :, sv, :], eo)
    + np.einsum("Bsim, Atam, i -> ABtsai", S_1_mo[:, :, so, :], S_1_mo[:, :, sv, :], eo)
    - 0.5 * Ax0_Core(sv, so, so, so)(S_2_mo[:, :, :, :, so, so])
    + Ax0_Core(sv, so, so, so)(np.einsum("Atkm, Bslm -> ABtskl", S_1_mo[:, :, so, :], S_1_mo[:, :, so, :]))
    - Ax0_Core(sv, so, so, so)(np.einsum("Atkb, Bslb -> ABtskl", U_1_ov, U_1_ov))
    # line 2
    + np.einsum("Atka, Bski -> ABtsai", U_1_ov, F_1_mo[:, :, so, so])
    + np.einsum("Atbi, Bsba -> ABtsai", U_1_vo, F_1_mo[:, :, sv, sv])
    + np.einsum("Bska, Atki -> ABtsai", U_1_ov, F_1_mo[:, :, so, so])
    + np.einsum("Bsbi, Atba -> ABtsai", U_1_vo, F_1_mo[:, :, sv, sv])
    # line 4
    + Ax0_Core(sv, so, sv, sv)(np.einsum("Atbk, Bsck -> ABtsbc", U_1_vo, U_1_vo))
    # line 5
    + np.einsum("Atka, Bski -> ABtsai", U_1_ov, Ax0_Core(so, so, sa, so)(U_pi_fake))
    + np.einsum("Bska, Atki -> ABtsai", U_1_ov, Ax0_Core(so, so, sa, so)(U_pi_fake))
    # line 6
    + np.einsum("Atbi, Bsba -> ABtsai", U_1_vo, Ax0_Core(sv, sv, sa, so)(U_pi_fake))
    + np.einsum("Bsbi, Atba -> ABtsai", U_1_vo, Ax0_Core(sv, sv, sa, so)(U_pi_fake))
    # line 7
    + Ax1_Core(sv, so, sa, so)(U_pi_fake)
    + Ax1_Core(sv, so, sa, so)(U_pi_fake).transpose((1, 0, 3, 2, 4, 5))
# -- Unsafe
    # line 1
    - np.einsum("Atik, Bsak, i -> ABtsai", U_1[:, :, so, so], U_1[:, :, sv, so], eo)                     # term 01
    - np.einsum("Atib, Bsab, i -> ABtsai", U_1[:, :, so, sv], U_1[:, :, sv, sv], eo)                     # term 02
    - np.einsum("Bsik, Atak, i -> ABtsai", U_1[:, :, so, so], U_1[:, :, sv, so], eo)                     # term 03
    - np.einsum("Bsib, Atab, i -> ABtsai", U_1[:, :, so, sv], U_1[:, :, sv, sv], eo)                     # term 04
    - Ax0_Core(sv, so, so, so)(np.einsum("Atkj, Bslj -> ABtskl", U_1[:, :, so, so], U_1[:, :, so, so]))  # term 05
    # line 2
    + np.einsum("Atba, Bsbi -> ABtsai", U_1[:, :, sv, sv], F_1_mo[:, :, sv, so])                         # term 06
    + np.einsum("Atki, Bska -> ABtsai", U_1[:, :, so, so], F_1_mo[:, :, so, sv])                         # term 07
    + np.einsum("Bsba, Atbi -> ABtsai", U_1[:, :, sv, sv], F_1_mo[:, :, sv, so])                         # term 08
    + np.einsum("Bski, Atka -> ABtsai", U_1[:, :, so, so], F_1_mo[:, :, so, sv])                         # term 09
    # line 3
    + np.einsum("Atba, Bsbi, b -> ABtsai", U_1[:, :, sv, sv], U_1[:, :, sv, so], ev)                     # term 10
    + np.einsum("Atka, Bski, k -> ABtsai", U_1[:, :, so, sv], U_1[:, :, so, so], eo)                     # term 11
    + np.einsum("Bsba, Atbi, b -> ABtsai", U_1[:, :, sv, sv], U_1[:, :, sv, so], ev)                     # term 12
    + np.einsum("Bska, Atki, k -> ABtsai", U_1[:, :, so, sv], U_1[:, :, so, so], eo)                     # term 13
    # line 4
    + Ax0_Core(sv, so, so, sv)(np.einsum("Atjk, Bsbk -> ABtsjb", U_1[:, :, so, so], U_1[:, :, sv, so]))  # term 14
    + Ax0_Core(sv, so, sv, so)(np.einsum("Atbk, Bsjk -> ABtsbj", U_1[:, :, sv, so], U_1[:, :, so, so]))  # term 15
    + Ax0_Core(sv, so, so, so)(np.einsum("Atkj, Bslj -> ABtskl", U_1[:, :, so, so], U_1[:, :, so, so]))  # term 16
    # line 5
    + np.einsum("Atba, Bsbi -> ABtsai", U_1[:, :, sv, sv], Ax0_Core(sv, so, sa, so)(U_1[:, :, :, so]))   # term 17
    + np.einsum("Bsba, Atbi -> ABtsai", U_1[:, :, sv, sv], Ax0_Core(sv, so, sa, so)(U_1[:, :, :, so]))   # term 18
    # line 6
    + np.einsum("Atki, Bska -> ABtsai", U_1[:, :, so, so], Ax0_Core(so, sv, sa, so)(U_1[:, :, :, so]))   # term 19
    + np.einsum("Bski, Atka -> ABtsai", U_1[:, :, so, so], Ax0_Core(so, sv, sa, so)(U_1[:, :, :, so]))   # term 20
    ,
    hfh.B_2_vo
)

**不安全项抵消：正负相消**

首先，`05` 与 `16` 是最为简单的正负相消．

In [ ]:
np.allclose(
    - Ax0_Core(sv, so, so, so)(np.einsum("Atkj, Bslj -> ABtskl", U_1[:, :, so, so], U_1[:, :, so, so]))  # term 05
    + Ax0_Core(sv, so, so, so)(np.einsum("Atkj, Bslj -> ABtskl", U_1[:, :, so, so], U_1[:, :, so, so])), # term 16
    0
)

**不安全项抵消：CP-HF 方程**

其次，根据 CP-HF 方程

$$
F_{ai}^{A_t} - S_{ai}^{A_t} \varepsilon_i + A_{ai, qj} U_{qj}^{A_t} + (\varepsilon_a - \varepsilon_i) U_{ai}^{A_t} = 0
$$

我们可以约去一些项；但同时也会多一些项．由于 CP-HF 精度的问题，这里的 `np.allclose` 并不是那么准确．

* `06` $U_{ba}^{A_t} F_{bi}^{B_s}$ `17` $U_{ba}^{A_t} A_{bi, qk} U_{qk}^{B_s}$ `04` $- U_{ib}^{B_s} U_{ab}^{A_t} \varepsilon_i$ `10` $U_{ba}^{A_t} U_{bi}^{B_s} \varepsilon_b$ 相加所得为安全项 $U_{ib}^{B_s} S_{ba}^{A_t} \varepsilon_i$

In [ ]:
np.allclose(
    + np.einsum("Atba, Bsbi -> ABtsai", U_1[:, :, sv, sv], F_1_mo[:, :, sv, so])                         # term 06
    + np.einsum("Atba, Bsbi -> ABtsai", U_1[:, :, sv, sv], Ax0_Core(sv, so, sa, so)(U_1[:, :, :, so]))   # term 17
    - np.einsum("Bsib, Atab, i -> ABtsai", U_1[:, :, so, sv], U_1[:, :, sv, sv], eo)                     # term 04
    + np.einsum("Atba, Bsbi, b -> ABtsai", U_1[:, :, sv, sv], U_1[:, :, sv, so], ev),                    # term 10
    + np.einsum("Bsib, Atba, i -> ABtsai", U_1_ov, S_1_mo[:, :, sv, sv], eo),                            # safe
    atol=1e-5
)

`08` $U_{ba}^{B_s} F_{bi}^{A_t}$ `18` $U_{ba}^{B_s} A_{bi, qk} U_{qk}^{A_t}$ `02` $- U_{ib}^{A_t} U_{ab}^{B_s} \varepsilon_i$ `12` $U_{ba}^{B_s} U_{bi}^{A_t} \varepsilon_b$ 相加所得为安全项 $U_{ib}^{A_t} S_{ba}^{B_s} \varepsilon_i$

In [ ]:
np.allclose(
    + np.einsum("Bsba, Atbi -> ABtsai", U_1[:, :, sv, sv], F_1_mo[:, :, sv, so])                         # term 08
    + np.einsum("Bsba, Atbi -> ABtsai", U_1[:, :, sv, sv], Ax0_Core(sv, so, sa, so)(U_1[:, :, :, so]))   # term 18
    - np.einsum("Atib, Bsab, i -> ABtsai", U_1[:, :, so, sv], U_1[:, :, sv, sv], eo)                     # term 02
    + np.einsum("Bsba, Atbi, b -> ABtsai", U_1[:, :, sv, sv], U_1[:, :, sv, so], ev),                    # term 12
    + np.einsum("Atib, Bsba, i -> ABtsai", U_1_ov, S_1_mo[:, :, sv, sv], eo),                            # safe
    atol=1e-5
)

`07` $U_{ki}^{A_t} F_{ka}^{B_s}$ `19` $U_{ki}^{A_t} A_{ka, qk} U_{qk}^{B_s}$ `13` $U_{ka}^{B_s} U_{ki}^{A_t} \varepsilon_k$ 相加所得为非安全项

* `21` $- U_{ki}^{A_t} U_{ak}^{B_s} \varepsilon_a$

In [ ]:
np.allclose(
    + np.einsum("Atki, Bska -> ABtsai", U_1[:, :, so, so], F_1_mo[:, :, so, sv])                         # term 07
    + np.einsum("Atki, Bska -> ABtsai", U_1[:, :, so, so], Ax0_Core(so, sv, sa, so)(U_1[:, :, :, so]))   # term 19
    + np.einsum("Bska, Atki, k -> ABtsai", U_1[:, :, so, sv], U_1[:, :, so, so], eo),                    # term 13
    - np.einsum("Atki, Bsak, a -> ABtsai", U_1[:, :, so, so], U_1[:, :, sv, so], ev),                    # term 21
    atol=1e-5
)

`09` $U_{ki}^{B_s} F_{ka}^{A_t}$ `20` $U_{ki}^{B_s} A_{ka, qk} U_{qk}^{A_t}$ `11` $U_{ka}^{A_t} U_{ki}^{B_s} \varepsilon_k$ 相加所得为非安全项

* `22` $- U_{ki}^{B_s} U_{ak}^{A_t} \varepsilon_a$

In [ ]:
np.allclose(
    + np.einsum("Bski, Atka -> ABtsai", U_1[:, :, so, so], F_1_mo[:, :, so, sv])                         # term 09
    + np.einsum("Bski, Atka -> ABtsai", U_1[:, :, so, so], Ax0_Core(so, sv, sa, so)(U_1[:, :, :, so]))   # term 20
    + np.einsum("Atka, Bski, k -> ABtsai", U_1[:, :, so, sv], U_1[:, :, so, so], eo),                    # term 11
    - np.einsum("Bski, Atak, a -> ABtsai", U_1[:, :, so, so], U_1[:, :, sv, so], ev),                    # term 22
    atol=1e-5
)

**最后总结**

现在我们总结一下 $B_{ai}^{A_t B_s}$ 中还剩下的非安全项；这些项在 $B_{ai}^{A_t B_s}$ 之内应当难以再被简化或者转为安全项，需要通过外部的手段将其消除．

在这里，我们暂时不对安全项作进一步简化与性能优化．我们等到生成 $\mathscr{B}_{ai}^{A_t B_s}$ 后，一并进行．

现在还剩余的非安全项：

* `01` $- U_{ik}^{A_t} U_{ak}^{B_s} \varepsilon_i$

* `03` $- U_{ik}^{B_s} U_{ak}^{A_t} \varepsilon_i$

* `14` $A_{ai, jb} U_{jk}^{A_t} U_{bk}^{B_s}$

* `15` $A_{ai, bj} U_{bk}^{A_t} U_{jk}^{B_s}$

* `21` $- U_{ki}^{A_t} U_{ak}^{B_s} \varepsilon_a$

* `22` $- U_{ki}^{B_s} U_{ak}^{A_t} \varepsilon_a$

In [ ]:
np.allclose(
# -- Safe
    # line 1
    np.einsum("ABtsuv, ua, vi -> ABtsai", hfh.F_2_ao, hfh.Cv, hfh.Co)
    - np.einsum("ABtsai, i -> ABtsai", S_2_mo[:, :, :, :, sv, so], eo)
    + np.einsum("Atim, Bsam, i -> ABtsai", S_1_mo[:, :, so, :], S_1_mo[:, :, sv, :], eo)
    + np.einsum("Bsim, Atam, i -> ABtsai", S_1_mo[:, :, so, :], S_1_mo[:, :, sv, :], eo)
    - 0.5 * Ax0_Core(sv, so, so, so)(S_2_mo[:, :, :, :, so, so])
    + Ax0_Core(sv, so, so, so)(np.einsum("Atkm, Bslm -> ABtskl", S_1_mo[:, :, so, :], S_1_mo[:, :, so, :]))
    - Ax0_Core(sv, so, so, so)(np.einsum("Atkb, Bslb -> ABtskl", U_1_ov, U_1_ov))
    # line 2
    + np.einsum("Atka, Bski -> ABtsai", U_1_ov, F_1_mo[:, :, so, so])
    + np.einsum("Atbi, Bsba -> ABtsai", U_1_vo, F_1_mo[:, :, sv, sv])
    + np.einsum("Bska, Atki -> ABtsai", U_1_ov, F_1_mo[:, :, so, so])
    + np.einsum("Bsbi, Atba -> ABtsai", U_1_vo, F_1_mo[:, :, sv, sv])
    # line 4
    + Ax0_Core(sv, so, sv, sv)(np.einsum("Atbk, Bsck -> ABtsbc", U_1_vo, U_1_vo))
    # line 5
    + np.einsum("Atka, Bski -> ABtsai", U_1_ov, Ax0_Core(so, so, sa, so)(U_pi_fake))
    + np.einsum("Bska, Atki -> ABtsai", U_1_ov, Ax0_Core(so, so, sa, so)(U_pi_fake))
    # line 6
    + np.einsum("Atbi, Bsba -> ABtsai", U_1_vo, Ax0_Core(sv, sv, sa, so)(U_pi_fake))
    + np.einsum("Bsbi, Atba -> ABtsai", U_1_vo, Ax0_Core(sv, sv, sa, so)(U_pi_fake))
    # line 7
    + Ax1_Core(sv, so, sa, so)(U_pi_fake)
    + Ax1_Core(sv, so, sa, so)(U_pi_fake).transpose((1, 0, 3, 2, 4, 5))
    # Obtained by unsafe-elimination
    + np.einsum("Bsib, Atba, i -> ABtsai", U_1_ov, S_1_mo[:, :, sv, sv], eo)
    + np.einsum("Atib, Bsba, i -> ABtsai", U_1_ov, S_1_mo[:, :, sv, sv], eo)
# -- Unsafe
    - np.einsum("Atik, Bsak, i -> ABtsai", U_1[:, :, so, so], U_1[:, :, sv, so], eo)                     # term 01
    - np.einsum("Bsik, Atak, i -> ABtsai", U_1[:, :, so, so], U_1[:, :, sv, so], eo)                     # term 03
    + Ax0_Core(sv, so, so, sv)(np.einsum("Atjk, Bsbk -> ABtsjb", U_1[:, :, so, so], U_1[:, :, sv, so]))  # term 14
    + Ax0_Core(sv, so, sv, so)(np.einsum("Atbk, Bsjk -> ABtsbj", U_1[:, :, sv, so], U_1[:, :, so, so]))  # term 15
    - np.einsum("Atki, Bsak, a -> ABtsai", U_1[:, :, so, so], U_1[:, :, sv, so], ev)                     # term 21
    - np.einsum("Bski, Atak, a -> ABtsai", U_1[:, :, so, so], U_1[:, :, sv, so], ev)                     # term 22
    ,
    hfh.B_2_vo,
    atol=2e-5
)

#### $\mathscr{B}_{ai}^{A_t B_s}$ 构建

我们已经花了太长时间在 $B_{ai}^{A_t B_s}$ 安全项分析了；现在我们回到我们本来的目标．

$$
\mathscr{B}_{ai}^{A_t B_s} = A_{ai, bj}' (U_{bj}^{A_t B_s} + U_{bk}^{A_t} U_{jk}^{B_s} + U_{bk}^{B_s} U_{jk}^{A_t}) = A_{ai, bj}' (U_{bk}^{A_t} U_{jk}^{B_s} + U_{bk}^{B_s} U_{jk}^{A_t}) + B_{ai}^{A_t}
$$

我们注意到定义 $A_{ai, bj}' = - A_{ai, bj} - \delta_{ab} \delta_{ij} (\varepsilon_b - \varepsilon_j))$，因此

\begin{align*}
  &\quad A_{ai, bj}' (U_{bk}^{A_t} U_{jk}^{B_s} + U_{bk}^{B_s} U_{jk}^{A_t}) \\
  &= - A_{ai, bj} U_{bk}^{A_t} U_{jk}^{B_s} - \varepsilon_a U_{ak}^{A_t} U_{ik}^{B_s} + \varepsilon_i U_{ak}^{A_t} U_{ik}^{B_s} \\
  &\quad \mathrel- A_{ai, bj} U_{bk}^{B_s} U_{jk}^{A_t} - \varepsilon_a U_{ak}^{B_s} U_{ik}^{A_t} + \varepsilon_i U_{ak}^{B_s} U_{ik}^{A_t} \\
  &= - A_{ai, bj} U_{bk}^{A_t} U_{jk}^{B_s} + \varepsilon_a U_{ak}^{A_t} U_{ki}^{B_s} + \varepsilon_i U_{ak}^{A_t} U_{ik}^{B_s} \\
  &\quad \mathrel- A_{ai, bj} U_{bk}^{B_s} U_{jk}^{A_t} + \varepsilon_a U_{ak}^{B_s} U_{ki}^{A_t} + \varepsilon_i U_{ak}^{B_s} U_{ik}^{A_t} \\
  &\quad \mathrel+ \varepsilon_a U_{ak}^{A_t} S_{ki}^{B_s} + \varepsilon_a U_{ak}^{B_s} S_{ki}^{A_t}
\end{align*}

我们会注意到，上式最后一个等号后的前六项分别对应非安全项 `15`, `22`, `03`, `14`, `21`, `01` 的相反数；因此，刚好可以与 $B_{ai}^{A_t}$ 中的所有非安全项抵消．最后两项是安全项．因此，我们可以将 $\mathscr{B}_{ai}^{A_t B_s}$ 代码生成如下：

In [ ]:
B_2_new_vo = (
# -- from B_2_vo
    # line 1
    np.einsum("ABtsuv, ua, vi -> ABtsai", hfh.F_2_ao, hfh.Cv, hfh.Co)
    - np.einsum("ABtsai, i -> ABtsai", S_2_mo[:, :, :, :, sv, so], eo)
    + np.einsum("Atim, Bsam, i -> ABtsai", S_1_mo[:, :, so, :], S_1_mo[:, :, sv, :], eo)
    + np.einsum("Bsim, Atam, i -> ABtsai", S_1_mo[:, :, so, :], S_1_mo[:, :, sv, :], eo)
    - 0.5 * Ax0_Core(sv, so, so, so)(S_2_mo[:, :, :, :, so, so])
    + Ax0_Core(sv, so, so, so)(np.einsum("Atkm, Bslm -> ABtskl", S_1_mo[:, :, so, :], S_1_mo[:, :, so, :]))
    - Ax0_Core(sv, so, so, so)(np.einsum("Atkb, Bslb -> ABtskl", U_1_ov, U_1_ov))
    # line 2
    + np.einsum("Atka, Bski -> ABtsai", U_1_ov, F_1_mo[:, :, so, so])
    + np.einsum("Atbi, Bsba -> ABtsai", U_1_vo, F_1_mo[:, :, sv, sv])
    + np.einsum("Bska, Atki -> ABtsai", U_1_ov, F_1_mo[:, :, so, so])
    + np.einsum("Bsbi, Atba -> ABtsai", U_1_vo, F_1_mo[:, :, sv, sv])
    # line 4
    + Ax0_Core(sv, so, sv, sv)(np.einsum("Atbk, Bsck -> ABtsbc", U_1_vo, U_1_vo))
    # line 5
    + np.einsum("Atka, Bski -> ABtsai", U_1_ov, Ax0_Core(so, so, sa, so)(U_pi_fake))
    + np.einsum("Bska, Atki -> ABtsai", U_1_ov, Ax0_Core(so, so, sa, so)(U_pi_fake))
    # line 6
    + np.einsum("Atbi, Bsba -> ABtsai", U_1_vo, Ax0_Core(sv, sv, sa, so)(U_pi_fake))
    + np.einsum("Bsbi, Atba -> ABtsai", U_1_vo, Ax0_Core(sv, sv, sa, so)(U_pi_fake))
    # line 7
    + Ax1_Core(sv, so, sa, so)(U_pi_fake)
    + Ax1_Core(sv, so, sa, so)(U_pi_fake).transpose((1, 0, 3, 2, 4, 5))
    # Obtained by unsafe-elimination
    + np.einsum("Bsib, Atba, i -> ABtsai", U_1_ov, S_1_mo[:, :, sv, sv], eo)
    + np.einsum("Atib, Bsba, i -> ABtsai", U_1_ov, S_1_mo[:, :, sv, sv], eo)
# -- from AUU
    + np.einsum("Atak, Bski, a -> ABtsai", U_1_vo, S_1_mo[:, :, so, so], ev)
    + np.einsum("Bsak, Atki, a -> ABtsai", U_1_vo, S_1_mo[:, :, so, so], ev)
)

我们将这最后的安全项的导出结果归为 `E_UU_safe_4`；这直接由 $Z_{bj} \mathscr{B}_{ai}^{A_t B_s}$ 获得．

In [ ]:
E_UU_safe_4 = 4 * np.einsum("ai, ABtsai -> ABts", Z, B_2_new_vo)

就此，所有 U-U 导数的非安全项已经全部抵消，只剩下安全项的贡献了．我们可以验证这个结论．

In [ ]:
np.allclose(E_UU_safe_1 + E_UU_safe_2 + E_UU_safe_3 + E_UU_safe_4, E_UU)

### U-U 部分：$\mathscr{B}_{ai}^{A_t B_s}$ 的优化

尽管我们会说，我们只需要生成一次 $\mathscr{B}_{ai}^{A_t B_s}$ 即可计算二阶梯度的 U-U 的贡献的一部分，但这部分的计算量相当大，我们有必要对其作最大程度的简化．刚才我们没有详细记录安全项的编号，因此我们这里就试着从代码反推公式，并且以此来优化代码．

最好想的代码优化方法是利用 $\mathscr{B}_{ai}^{A_t B_s}$ 关于 $A_t, B_s$ 的对称性；这不论对于代码还是公式表达都会是极大的简便：

In [ ]:
B_2_new_vo_ = (
# -- from B_2_vo
    # line 1
    + 0.5 * np.einsum("ABtsuv, ua, vi -> ABtsai", hfh.F_2_ao, hfh.Cv, hfh.Co)
    - 0.5 * np.einsum("ABtsai, i -> ABtsai", S_2_mo[:, :, :, :, sv, so], eo)
    + np.einsum("Atim, Bsam, i -> ABtsai", S_1_mo[:, :, so, :], S_1_mo[:, :, sv, :], eo)
    - 0.25 * Ax0_Core(sv, so, so, so)(S_2_mo[:, :, :, :, so, so])
    + 0.5 * Ax0_Core(sv, so, so, so)(np.einsum("Atkm, Bslm -> ABtskl", S_1_mo[:, :, so, :], S_1_mo[:, :, so, :]))
    - 0.5 * Ax0_Core(sv, so, so, so)(np.einsum("Atkb, Bslb -> ABtskl", U_1_ov, U_1_ov))
    # line 2
    + np.einsum("Atka, Bski -> ABtsai", U_1_ov, F_1_mo[:, :, so, so])
    + np.einsum("Atbi, Bsba -> ABtsai", U_1_vo, F_1_mo[:, :, sv, sv])
    # line 4
    + 0.5 * Ax0_Core(sv, so, sv, sv)(np.einsum("Atbk, Bsck -> ABtsbc", U_1_vo, U_1_vo))
    # line 5, 6
    + np.einsum("Atka, Bski -> ABtsai", U_1_ov, Ax0_Core(so, so, sa, so)(U_pi_fake))
    + np.einsum("Atbi, Bsba -> ABtsai", U_1_vo, Ax0_Core(sv, sv, sa, so)(U_pi_fake))
    # line 7
    + Ax1_Core(sv, so, sa, so)(U_pi_fake)
    # Obtained by unsafe-elimination
    + np.einsum("Bsib, Atba, i -> ABtsai", U_1_ov, S_1_mo[:, :, sv, sv], eo)
# -- from AUU
    + np.einsum("Atak, Bski, a -> ABtsai", U_1_vo, S_1_mo[:, :, so, so], ev)
)

B_2_new_vo_ += B_2_new_vo_.transpose(1, 0, 3, 2, 4, 5)

np.allclose(B_2_new_vo_, B_2_new_vo)

随后我们对其中两处使用到 `Ax0_Core` 处进行项归并．这里用到 `Ax0_Core` 中，可以在 `si, sj` 处传入 None 以使 `Ax0_Core` 返回 $A_{\mu \nu, rs} X_{rs}^{A_t}$．其中，$A_{pq, rs} X_{rs}^{A_t} = C_{\mu p} C_{\nu q} A_{\mu \nu, rs} X_{rs}^{A_t}$．

In [ ]:
B_2_tmp_1 = (
    hfh.Co @ (
        - 0.25 * S_2_mo[:, :, :, :, so, so]
        + 0.5 * np.einsum("Atkm, Bslm -> ABtskl", S_1_mo[:, :, so, :], S_1_mo[:, :, so, :])
        - 0.5 * np.einsum("Atkb, Bslb -> ABtskl", U_1_ov, U_1_ov)
    ) @ hfh.Co.T
    + 0.5 * hfh.Cv @ np.einsum("Atbk, Bsck -> ABtsbc", U_1_vo, U_1_vo) @ hfh.Cv.T
)
B_2_tmp_2 = Ax0_Core(None, None, sa, so)(U_pi_fake)

B_2_new_vo_ = (
    + 0.5 * np.einsum("ABtsuv, ua, vi -> ABtsai", hfh.F_2_ao, hfh.Cv, hfh.Co)
    - 0.5 * np.einsum("ABtsai, i -> ABtsai", S_2_mo[:, :, :, :, sv, so], eo)
    + np.einsum("Atim, Bsam, i -> ABtsai", S_1_mo[:, :, so, :], S_1_mo[:, :, sv, :], eo)
    + np.einsum("Atka, Bski -> ABtsai", U_1_ov, F_1_mo[:, :, so, so])
    + np.einsum("Atbi, Bsba -> ABtsai", U_1_vo, F_1_mo[:, :, sv, sv])
    + Ax0_Core(sv, so, None, None)(B_2_tmp_1)
    + np.einsum("Atka, uk, vi, Bsuv -> ABtsai", U_1_ov, hfh.Co, hfh.Co, B_2_tmp_2)
    + np.einsum("Atbi, ub, va, Bsuv -> ABtsai", U_1_vo, hfh.Cv, hfh.Cv, B_2_tmp_2)
    + Ax1_Core(sv, so, sa, so)(U_pi_fake)
    + np.einsum("Bsib, Atba, i -> ABtsai", U_1_ov, S_1_mo[:, :, sv, sv], eo)
    + np.einsum("Atak, Bski, a -> ABtsai", U_1_vo, S_1_mo[:, :, so, so], ev)
)

B_2_new_vo_ += B_2_new_vo_.transpose(1, 0, 3, 2, 4, 5)

np.allclose(B_2_new_vo_, B_2_new_vo)

预计上述过程中，语句 `B_2_tmp_2` 偏大 (相当于生成一次 $F_{\mu \nu}^{A_t}$)，而语句 `Ax0_Core(sv, so, None, None)(B_2_tmp_1)` 与 `Ax1_Core(sv, so, sa, so)(U_pi_fake)` 相当于生成一次 $F_{\mu \nu}^{A_t B_s}$ 的计算量．我们将它们控制到整个 $\mathscr{B}_{ai}^{A_t B_s}$ 计算过程中只生成一次．剩余的张量缩并相对来说计算量要小得多，因为这些张量都是已经生成过的了．